Homework Week 5 <br>
MSDS 600<br>
Bill Beemer<br>

Import Block

In [24]:
import pandas as pd
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model
import pickle

%matplotlib inline


Build dataframe from Prepped Churn Data

In [25]:
df = pd.read_csv('data/prepped_churn_data.csv', index_col='customerID')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,
7590-VHVEG,1,0,1,1,29.85,29.85,0
5575-GNVDE,34,1,12,2,56.95,1889.50,0
3668-QPYBK,2,1,1,2,53.85,108.15,1
7795-CFOCW,45,0,12,3,42.30,1840.75,0
9237-HQITU,2,1,1,1,70.70,151.65,1
...,...,...,...,...,...,...,...
6840-RESVB,24,1,12,2,84.80,1990.50,0
2234-XADUH,72,1,12,4,103.20,7362.90,0
4801-JZAZL,11,0,1,1,29.60,346.45,0


Setup Auto ML

In [26]:
automl = setup(df, target='Churn')

,Description,Value
0,Session id,2966
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7032, 7)"
4,Transformed data shape,"(7032, 7)"
5,Transformed train set shape,"(4922, 7)"
6,Transformed test set shape,"(2110, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


The below is not working - can't see different elements within the automl set

In [27]:
# automl[6]

Run Auto ML for best model

In [28]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7942,0.8329,0.5107,0.6415,0.5682,0.4355,0.4407,0.1900
ridge,Ridge Classifier,0.7889,0.0000,0.4350,0.6550,0.5223,0.3940,0.4078,0.0060
lda,Linear Discriminant Analysis,0.7885,0.8180,0.4893,0.6314,0.5509,0.4156,0.4216,0.0060
ada,Ada Boost Classifier,0.7883,0.8334,0.4817,0.6341,0.5468,0.4122,0.4192,0.0300
gbc,Gradient Boosting Classifier,0.7869,0.8361,0.4763,0.6310,0.5419,0.4069,0.4142,0.0800
lightgbm,Light Gradient Boosting Machine,0.7824,0.8269,0.5016,0.6106,0.5498,0.4084,0.4124,0.2550
rf,Random Forest Classifier,0.7607,0.7944,0.4664,0.5599,0.5085,0.3522,0.3549,0.0780
knn,K Neighbors Classifier,0.7560,0.7541,0.4366,0.5512,0.4869,0.3298,0.3338,0.0110
svm,SVM - Linear Kernel,0.7552,0.0000,0.4020,0.6436,0.4506,0.3151,0.3545,0.0080
et,Extra Trees Classifier,0.7529,0.7706,0.4702,0.5415,0.5028,0.3396,0.3414,0.0620


Display Best

In [29]:
print(type(best_model))
best_model

<class 'sklearn.linear_model._logistic.LogisticRegression'>


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=2966, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Analysis::<br>
It looks like LR is the best, following closely by the next top 5 other models (roughly 78%)

Making predictions using best model

In [41]:
print(df.iloc[-2:-1].shape)
predict_model(best_model, df.iloc[-2:-1])

(1, 7)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,0,1.0000,1.0000,1.0000,nan,0.0000


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,prediction_label,prediction_score
customerID,,,,,,,,,
8361-LTMKD,4,1,1,2,74.400002,306.600006,1,1,0.5758


Take a look at results for entire data frame

In [37]:
predictions = automl.predict_model(best_model, data=df)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.7956,0.8356,0.5136,0.6452,0.5719,0.4400,0.4449


In [38]:
predictions.head()

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,prediction_label,prediction_score
customerID,,,,,,,,,
7590-VHVEG,1,0,1,1,29.850000,29.850000,0,1,0.5343
5575-GNVDE,34,1,12,2,56.950001,1889.500000,0,0,0.9236
3668-QPYBK,2,1,1,2,53.849998,108.150002,1,0,0.5469
7795-CFOCW,45,0,12,3,42.299999,1840.750000,0,0,0.9504
9237-HQITU,2,1,1,1,70.699997,151.649994,1,1,0.6033


Analysis::<br>
I find it very interesting that the score is roughly 58% accurate - not great<br>
...and here we can see that play out, with getting the pattern above...<br>
wrong<br>
right<br>
wrong<br>
right<br>
right<br>
That is only 60% right when only viewing the head of the dataframe

Save model for use with Python

In [40]:
save_model(best_model, 'LR_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'PhoneService',
                                              'Contract', 'PaymentMethod',
                                              'MonthlyCharges', 'TotalCharges'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean',
                                                               verbose='deprecated'))),
                 ('...
                                        

Set up a pickle file, save and then load the pickle file

In [32]:
with open('LR_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

with open('LR_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

new_data = df.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

array([1], dtype=int8)

Load saved Pycaret model and make predictions

In [33]:
loaded_lr_model = load_model('LR_model')

Transformation Pipeline and Model Successfully Loaded


In [42]:
predict_model(loaded_lr_model, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,prediction_label,prediction_score
customerID,,,,,,,,
8361-LTMKD,4,1,1,2,74.400002,306.600006,1,0.5758


Run the prediction model using a Python library

In [35]:
%run homework_week5.py

Transformation Pipeline and Model Successfully Loaded


Predictions:
customerID
9305-CKSKC       Churn
1452-KNGVK       Churn
6723-OKKJM    No Churn
7832-POPKP    No Churn
6348-TACGU       Churn
Name: Churn_prediction, dtype: object


Analysis::<br>
Compared to correct churn or no churn - the above shows the following accuracy:<br>
right<br>
wrong<br>
right<br>
wrong<br>
wrong<br>
A meezly 40% accuracy - ugh!

### ANALYSIS
<br>
This weeks assignment was super frustrating with pycaret not playing nice with Python 3.11.  I really spun my wheels for some time.  Even after your lecture landed, those instructions did not fully solve the problem I was having.  SO that being said, I feel like, as far as analysis goes this week, I don't have the energy to really dive deep.<br>
<br>
Using git on a mac is frustrating due to the filesystem constently adding icon files into every dir - and though my git ignore contains a line to ignore them, they still keep createing headaches - I am well aware of how to resolve this issue, it just has been a while since I needed to use git on my mac.  Professionally I use git every day on Kubuntu 20, including coding in Python with VS Code.<br><br>
Hope everything above is up to snuff... thank you!
